In [1]:
import pandas as pd
import numpy as np 
import glob
from tqdm import tqdm_notebook
import os, subprocess


In [2]:
# Read in data
data = {}
for file in tqdm_notebook(sorted(glob.glob("*.parquet"))):
    name = file.split(".")[0]
    data[name]=pd.read_parquet(file)
data.pop('gene_index')

,gene,Description,Chromosome.Name,Gene.Start..bp.,Gene.End..bp.,Associated.Gene.Name,Gene.Biotype
0,ENSG00000223116,,13,23551994,23552136,AL157931.1,miRNA
1,ENSG00000233440,high mobility group AT-hook 1 pseudogene 6 [So...,13,23708313,23708703,HMGA1P6,pseudogene
2,ENSG00000207157,"RNA, Ro-associated Y3 pseudogene 4 [Source:HGN...",13,23726725,23726825,RNY3P4,misc_RNA
3,ENSG00000229483,long intergenic non-protein coding RNA 362 [So...,13,23743974,23744736,LINC00362,lincRNA
4,ENSG00000252952,"RNA, U6 small nuclear 58, pseudogene [Source:H...",13,23791571,23791673,RNU6-58P,snRNA
5,ENSG00000235205,TatD DNase domain containing 2 pseudogene 3 [S...,13,23817659,23821323,TATDN2P3,pseudogene
6,ENSG00000232849,long intergenic non-protein coding RNA 363 [So...,13,93708910,93710179,LINC00363,lincRNA
7,ENSG00000236803,SDA1 domain containing 1 pseudogene 4 [Source:...,13,23841645,23843289,SDAD1P4,pseudogene
8,ENSG00000232163,"ribosomal protein, large, P1 pseudogene 13 [So...",13,23947562,23948200,RPLP1P13,pseudogene
9,ENSG00000252365,Small nucleolar RNA SNORD22 [Source:RFAM;Acc:R...,13,94021100,94021213,SNORD22,snoRNA


In [3]:
# load genes
gene_descriptions = pd.read_csv('gene_list.csv')
names = gene_descriptions['Approved Symbol'].values.astype(str)
all_genes = pd.read_parquet('gene_index.parquet')

In [4]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [5]:
from matplotlib import pyplot as plt

In [6]:
def means(cluster):
    df = pd.DataFrame(columns=["Gene", "Mean","Stdev"])
    for name in sorted(names):
        b=all_genes[all_genes['Associated.Gene.Name']==name]['gene']
        x=data[cluster][data[cluster]['genes'].isin(b.values)]
        if x.values[:,1:].astype('float32').mean()!= 0 and not np.isnan(x.values[:,1:].astype('float32').mean()):
            df=df.append({"Gene":name,"Mean":x.values[:,1:].astype('float32').mean(),"Stdev":x.values[:,1:].astype('float32').std()}, ignore_index=True)
    print(df.to_string())

In [7]:
interact(means,cluster=sorted(data.keys()))

interactive(children=(Dropdown(description='cluster', options=('alveolar1', 'alveolar2', 'alveolar3', 'alveola…

<function __main__.means(cluster)>

In [41]:

def hist_cell(cell_type,gene):
    plt.figure(dpi=150)
    b=all_genes[all_genes['Associated.Gene.Name']==gene]['gene']
    new_cats=[]
    for key in sorted(data.keys()):
        if cell_type in key:
            new_cats.append(key)
    counts = []
    bins=np.linspace(-0.1,3.9,100)
    for cat in new_cats:
        curr_set = data[cat]
        vals = curr_set[curr_set['genes'].isin(b.values)]
        counts.append(vals.values[:,1:].astype('float32').flatten())
    plt.ylim(0,50)
    plt.hist(counts,bins,label=new_cats)
    plt.title(cell_type+"-"+gene)
    plt.legend()

In [42]:
%matplotlib notebook
interact(hist_cell, cell_type=set([val[:-1] for val in sorted(data.keys())]), gene=sorted(names))

interactive(children=(Dropdown(description='cell_type', options=('t_cell', 'b_cell', 'ec', 'fibro', 'alveolar'…

<function __main__.hist_cell(cell_type, gene)>